# Similarity Matching

In [1]:
%run setup.py

In [103]:
# Load data
df_Invited = pd.read_csv(os.path.join(filepath,'factInvited.csv'))
df_Invited['Source'] = "Invited"
df_Proceedings = pd.read_csv(os.path.join(filepath,'factProceedings.csv'))
df_Proceedings['Source'] = "Proceedings"
# Append data to one dataframe
df = df_Invited.append(df_Proceedings).reset_index(drop=True)
df = df[df['Productivity'] != 0]
df.head()

,Full name,Year,Conference (short),First name,Middle name,Last name,gender,Paper Count,Max Paper Count,First year paper,Age,Conference (long),Main Topic,Productivity,Source
0,Gary L. Baldwi,2004,ASPDAC,Gary,L.,Baldwi,M,4,4,1962,42,Asia and South Pacific Design Automation Confe...,Computer Architecture,0.095238,Invited
1,Rudy Lauwereins,2004,ASPDAC,Rudy,NaN,Lauwereins,M,112,196,1987,17,Asia and South Pacific Design Automation Confe...,Computer Architecture,6.588235,Invited
2,Rajeev Madhavan,2005,ASPDAC,Rajeev,NaN,Madhavan,M,5,5,1994,11,Asia and South Pacific Design Automation Confe...,Computer Architecture,0.454545,Invited
3,Jan M. Rabaey,2005,ASPDAC,Jan,M.,Rabaey,M,146,321,1985,20,Asia and South Pacific Design Automation Confe...,Computer Architecture,7.300000,Invited
4,Alberto Sangiovanni-Vincentelli,2006,ASPDAC,Alberto,NaN,Sangiovanni-Vincentelli,M,457,689,1973,33,Asia and South Pacific Design Automation Confe...,Computer Architecture,13.848485,Invited


# Method 1

In [104]:
# Group the data by gender
grouped_data = df.groupby('gender')

# Calculate the cosine similarity between males and females
male_data = grouped_data.get_group('M')[['Paper Count', 'Age', 'Productivity']].reset_index()
female_data = grouped_data.get_group('F')[['Paper Count', 'Age', 'Productivity']].reset_index()
male_data_norm = (male_data - male_data.mean()) / male_data.std()
female_data_norm = (female_data - female_data.mean()) / female_data.std()
similarity_matrix = cosine_similarity(male_data_norm, female_data_norm)

df['Similar Females'] = ""

# List which males are similar to which females
for i, male in male_data.iterrows():
    similar_females = female_data.iloc[similarity_matrix[i].argsort()[::-1][:3]]
    similar_females_index = similar_females['index'].to_list() # get the index of similar females
    df.loc[male['index'], 'Similar Females'] = ", ".join(map(str, similar_females_index)) # assign to the new column

In [105]:
df.head()

,Full name,Year,Conference (short),First name,Middle name,Last name,gender,Paper Count,Max Paper Count,First year paper,Age,Conference (long),Main Topic,Productivity,Source,Similar Females
0,Gary L. Baldwi,2004,ASPDAC,Gary,L.,Baldwi,M,4,4,1962,42,Asia and South Pacific Design Automation Confe...,Computer Architecture,0.095238,Invited,"5878, 3985, 14365"
1,Rudy Lauwereins,2004,ASPDAC,Rudy,NaN,Lauwereins,M,112,196,1987,17,Asia and South Pacific Design Automation Confe...,Computer Architecture,6.588235,Invited,"3429, 6429, 4795"
2,Rajeev Madhavan,2005,ASPDAC,Rajeev,NaN,Madhavan,M,5,5,1994,11,Asia and South Pacific Design Automation Confe...,Computer Architecture,0.454545,Invited,"6049, 9596, 9128"
3,Jan M. Rabaey,2005,ASPDAC,Jan,M.,Rabaey,M,146,321,1985,20,Asia and South Pacific Design Automation Confe...,Computer Architecture,7.300000,Invited,"14534, 15063, 1251"
4,Alberto Sangiovanni-Vincentelli,2006,ASPDAC,Alberto,NaN,Sangiovanni-Vincentelli,M,457,689,1973,33,Asia and South Pacific Design Automation Confe...,Computer Architecture,13.848485,Invited,"497, 7532, 7578"


In [108]:
# List which males are similar to which females
for i, male in male_data.iterrows():
    similarities = similarity_matrix[i].argsort()[::-1][:3]
    similar_females = female_data.iloc[similarities]
    similarity_values = similarity_matrix[i][similarities]
    for k in range(len(similarities)):
        j = similar_females.index[k]
        similarity = similarity_values[k]
        print(f"Male speaker {i} is similar to female speaker {j} with a similarity index of {similarity:.2f}")

Male speaker 0 is similar to female speaker 898 with a similarity index of 1.00
Male speaker 0 is similar to female speaker 588 with a similarity index of 1.00
Male speaker 0 is similar to female speaker 2840 with a similarity index of 1.00
Male speaker 1 is similar to female speaker 505 with a similarity index of 1.00
Male speaker 1 is similar to female speaker 1006 with a similarity index of 1.00
Male speaker 1 is similar to female speaker 727 with a similarity index of 1.00
Male speaker 2 is similar to female speaker 936 with a similarity index of 1.00
Male speaker 2 is similar to female speaker 1662 with a similarity index of 1.00
Male speaker 2 is similar to female speaker 1555 with a similarity index of 1.00
Male speaker 3 is similar to female speaker 2876 with a similarity index of 1.00
Male speaker 3 is similar to female speaker 3010 with a similarity index of 1.00
Male speaker 3 is similar to female speaker 258 with a similarity index of 1.00
Male speaker 4 is similar to femal

KeyboardInterrupt: 

In [109]:
# List which males are similar to which females
for i, male in male_data.iterrows():
    similarities = similarity_matrix[i] == 1
    similar_females = female_data.iloc[similarities]
    similarity_values = similarity_matrix[i][similarities]
    for k in range(len(similarities)):
        j = similar_females.index[k]
        similarity = similarity_values[k]
        print(f"Male speaker {i} is similar to female speaker {j} with a similarity index of {similarity:.2f}")

IndexError: index 0 is out of bounds for axis 0 with size 0

In [97]:
'''
# Group the data by gender
grouped_data = df.groupby('gender')

# Calculate the cosine similarity between males and females
male_data = grouped_data.get_group('M')[['Paper Count', 'Age', 'Productivity']].reset_index()
female_data = grouped_data.get_group('F')[['Paper Count', 'Age', 'Productivity']].reset_index()
male_data_norm = (male_data - male_data.mean()) / male_data.std()
female_data_norm = (female_data - female_data.mean()) / female_data.std()
similarity_matrix = cosine_similarity(male_data_norm, female_data_norm)

# List which males are similar to which females
for i, male in male_data.iterrows():
    similar_females = female_data.iloc[similarity_matrix[i].argsort()[::-1][:3]]
    print(f'Male speaker {i} is similar to the following female speakers:')
    print(similar_females)
'''

"\n# Group the data by gender\ngrouped_data = df.groupby('gender')\n\n# Calculate the cosine similarity between males and females\nmale_data = grouped_data.get_group('M')[['Paper Count', 'Age', 'Productivity']].reset_index()\nfemale_data = grouped_data.get_group('F')[['Paper Count', 'Age', 'Productivity']].reset_index()\nmale_data_norm = (male_data - male_data.mean()) / male_data.std()\nfemale_data_norm = (female_data - female_data.mean()) / female_data.std()\nsimilarity_matrix = cosine_similarity(male_data_norm, female_data_norm)\n\n# List which males are similar to which females\nfor i, male in male_data.iterrows():\n    similar_females = female_data.iloc[similarity_matrix[i].argsort()[::-1][:3]]\n    print(f'Male speaker {i} is similar to the following female speakers:')\n    print(similar_females)\n"